In [17]:
import keras
from keras import layers
from keras.preprocessing import image

import os

import numpy as np

https://github.com/fchollet/deep-learning-with-python-notebooks/blob/master/8.5-introduction-to-gans.ipynb

Deep Convolution GAN - DCGAN

Структура сети:
* Сеть generator отображает векторы с формой (размерность_скрытого_простаранства) в изображения с формой (32, 32, 3).
Размерность изображения соответсвует размерам образцов из датасета CIFAR10
* Сеть discriminator отображает изображения с формой (32, 32, 3) в оценку вероятности того, что изображение является настоящим.
* Сеть gan объединяет генератор и дискриминатор gan(x) = discriminator(generator(x)). То есть сеть отображает скрытое простарнство векторов в оценку их реализма.

Дискриминатор будет обучен на изображениях с метками настоящее\поддельное. <br>
Для обучения генератора мы используем градиенты весов в отношении потерь модели gan. То есть на каждом шаге мы будем смещать веса генератора в направлении увеличения вероятности классификации изображений, декодированных как "настоящие". <br>
Проще говоря, будем учить генератор обманывать дискриминатор, повышая качество сгенерированных изображений.

**Генератор** <br>
Преобразует вектор из срытого пространства в изображение - кандидат.

In [6]:
latent_dim = 32
height = 32
width = 32
channels = 3

generator_input = keras.Input(shape = (latent_dim,))

# Преобразование входа в карту признаков 16х16 со 128 каналами
x = layers.Dense(128 * 16 * 16)(generator_input)
x = layers.LeakyReLU()(x)
x = layers.Reshape((16, 16, 128))(x)

# Увеличиваем разрешение с 16х16 до 32х32
x = layers.Conv2D(256, 5, padding = 'same')(x)
x = layers.LeakyReLU()(x)

x = layers.Conv2DTranspose(256, 4, strides = 2, padding = 'same')(x)
x = layers.LeakyReLU()(x)

x = layers.Conv2D(256, 5, padding = 'same')(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(256, 5, padding = 'same')(x)
x = layers.LeakyReLU()(x)

x = layers.Conv2D(channels, 7, activation = 'tanh', padding = 'same')(x)
generator = keras.models.Model(generator_input, x)
generator.summary()


Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 32)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 32768)             1081344   
_________________________________________________________________
leaky_re_lu_8 (LeakyReLU)    (None, 32768)             0         
_________________________________________________________________
reshape_4 (Reshape)          (None, 16, 16, 128)       0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 16, 16, 256)       819456    
_________________________________________________________________
leaky_re_lu_9 (LeakyReLU)    (None, 16, 16, 256)       0         
_________________________________________________________________
conv2d_transpose_2 (Conv2DTr (None, 32, 32, 256)       1048

**Дискриминатор**<br>
Принимает изображение-кандидат и относит его к одному из двух классов: "подделка" или "настоящее"

In [13]:
discriminator_input = layers.Input(shape = (height, width, channels))
x = layers.Conv2D(128, 3)(discriminator_input)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(128, 4, strides = 2)(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(128, 4, strides = 2)(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(128, 4, strides = 2)(x)
x = layers.LeakyReLU()(x)
x = layers.Flatten()(x)

x = layers.Dropout(0.4)(x)

x = layers.Dense(1, activation = 'sigmoid')(x)

discriminator = keras.models.Model(discriminator_input, x)
discriminator.summary()

discriminator_optimizer = keras.optimizers.RMSprop(lr = 0.008, 
                                                   clipvalue = 1.0, # градиентная обрезка (клиппинг)
                                                   decay = 1e-8) # затухание скорости обучения, для стабилизации
discriminator.compile(optimizer = discriminator_optimizer, 
                     loss = 'binary_crossentropy')

Model: "model_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_11 (InputLayer)        (None, 32, 32, 3)         0         
_________________________________________________________________
conv2d_34 (Conv2D)           (None, 30, 30, 128)       3584      
_________________________________________________________________
leaky_re_lu_37 (LeakyReLU)   (None, 30, 30, 128)       0         
_________________________________________________________________
conv2d_35 (Conv2D)           (None, 14, 14, 128)       262272    
_________________________________________________________________
leaky_re_lu_38 (LeakyReLU)   (None, 14, 14, 128)       0         
_________________________________________________________________
conv2d_36 (Conv2D)           (None, 6, 6, 128)         262272    
_________________________________________________________________
leaky_re_lu_39 (LeakyReLU)   (None, 6, 6, 128)         0   

**Состязательная сеть**<br>
Объединятет генератор и дискриминатор

In [16]:
discriminator.trainable = False #Заморозка весов дискриминатора

gan_input = keras.Input(shape = (latent_dim,))
gan_output = discriminator(generator(gan_input))
gan = keras.models.Model(gan_input, gan_output)

gan_optimizer = keras.optimizers.RMSprop(lr = 0.004, clipvalue = 1.0, decay = 1e-8)
gan.compile(optimizer = gan_optimizer, loss = 'binary_crossentropy')

**Загрузка данных и обучение**

In [18]:
(x_train, y_train), (_, _) = keras.datasets.cifar10.load_data()

x_train = x_train[y_train.flatten() == 2] # Выбираем класс изображений "птицы"

x_train = x_train.reshape((x_train.shape[0],) + (height, width, channels)).astype('float32')/255

170500096/170498071 [==============================] - 345s 2us/step


Цикл обучения:
* Извлечь случайные точки (случайный шум) из скрытого пространства.
* Создать изображение из генератора, использовав эти точки.
* Смешать сгенерированные изображения с настоящими.
* Обучить дискриминатор на этом наборе изображений, добавив к нему соответсвующие метки: "настоящее"/"подделка".
* Выбрать новые точки из сркытого пространства.
* 
